# Import Eppy and Set Work Environment

In this cell we import the EnergyPlus idd file, building idf file, and materials idf file that contains our material library (which we will require later on).

In [ ]:
! pip install eppy

In [ ]:
import sys
from eppy import modeleditor
from eppy.modeleditor import IDF
iddfile = r"C:\EnergyPlusV9-5-0\Energy+.idd" #change to your energy+ idd filepath
fname1 = r"C:\Users\GRA\OneDrive - Universidade de Lisboa\Desktop\IN+\C-Tech\EPPY\untitled.idf" #change to your idf file path
fname2 = r"C:\Users\GRA\OneDrive - Universidade de Lisboa\Desktop\IN+\C-Tech\EPPY\EPPY_Class\materials.idf" #change to your materials idf filepath
weather_file = r"C:\Users\GRA\OneDrive - Universidade de Lisboa\Desktop\IN+\C-Tech\EPPY\LISBOA\PRT_LB_Lisboa.Portela.AP.085360_TMYx.2004-2018.epw" #change to your weather file path


In [ ]:
from eppy import modeleditor

In [ ]:
IDF.setiddname(iddfile)
idf1 = IDF(fname1, weather_file)
materials = IDF(fname2, weather_file)

In this cell we import other packages required for Data visualization such as numpy and pandas

In [ ]:
! pip install numpy

In [ ]:
! pip install pandas

In [ ]:
! pip install matplotlib

In [ ]:
! pip install seaborn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Run IDF

In [ ]:
idf1.run()

# Data Visualization

To visualize the data we need to import esoreader, which reads the eso file that is returned by the E+ simulation

In [ ]:
! pip install esoreader

In [ ]:
import esoreader

In [ ]:
eso = esoreader.read_from_path(r"C:\Users\GRA\OneDrive - Universidade de Lisboa\Desktop\IN+\C-Tech\EPPY\EPPY_Class\eplusout.eso")

The eso file returns multiple python dictionaries with integer keys as seen above

In [ ]:
eso.dd.variables[119]

In [ ]:
eso.data[218]

As seen, we have a dictionary that yields the output timestep, building name, and output nme, for each key, and a dictionary that yields each key's respective data. Thus, we can create a dataframe from a dictionary for target keys that we want to visualize or study.

In [ ]:
eso.dd.variables.keys()

In [ ]:
targets = [7, 8, 119, 169, 218, 134, 183]
results = pd.DataFrame({eso.dd.variables[key][2]: eso.data[key] for key in eso.data.keys() if key in targets})

In [ ]:
results

In [ ]:
g = sns.lineplot(data=results[["Zone Ideal Loads Supply Air Total Cooling Energy",
                               "Zone Ideal Loads Supply Air Sensible Heating Energy"]], 
                 legend=True,
                 alpha=0.5,
                 linewidth=1)

sns.set(rc={"figure.figsize":(16, 8)})

# Creating zone Constructions and applying constructions to surfaces

EnergyPlus Zone constructions are IDF objects with layers as fields, and each layer has a material name that the software will call from the idf material objects. Thus, we need to isolate each material objects from our materials database IDF based on their type:

In [ ]:
mats = materials.idfobjects["MATERIAL"] #Opaque materials
window_mats = materials.idfobjects["WINDOWMATERIAL:GLAZING"] #Window materials - Glazings
window_airgaps_mats = materials.idfobjects["WINDOWMATERIAL:GAS"] #Window materials - Airgaps
airgap_mats = materials.idfobjects["MATERIAL:AIRGAP"] #airgap materials for opaque constructions

In [ ]:
full_mats = []
for i in [mats, window_mats, window_airgaps_mats, airgap_mats]:
    full_mats = np.append(full_mats, i)

In [ ]:
full_mats

We see that the opaque Material Names have spaces. While energy plus works with this, EPPY and python don't like spaces in definitions! lets remove the spaces from the strings and copy the idf objects to our zone idf

In [ ]:
for mat in full_mats:
    mat.Name = mat.Name.replace(" ", "")

In [ ]:
for mat in full_mats:
    idf1.copyidfobject(mat)

Now that we have copied our materials library to our idf, lets analyse the constructions in our zone and choose the materials for the construction we are going to create and apply to a wall. Thus, a wall construction

In [ ]:
construction = ["Reboco-2cm", "TijoloFurado_11", "XPS_4", "TijoloFurado_11", "Arg_Cimento_Clara_2.5", "Estuque_Claro_1.5"]

In [ ]:
constructions = idf1.idfobjects["CONSTRUCTION"]

In [ ]:
constructions

We see that each construction has a name, and fields for each layer, starting with our outside layer. Thus, we could add a construction like this:

In [ ]:
idf1.newidfobject("CONSTRUCTION",
                  Name="New_wall_construction",
                  Outside_Layer = construction[0],
                  Layer_2 = construction[1],
                  Layer_3 = construction[2],
                  Layer_4 = construction[3],
                  Layer_5 = construction[4],
                  Layer_6 = construction[5])

For all effects, EnergyPlus is a programming language, and it thas some flaws that we have to work through to implement certain solutions. In this case, a python array starts counting on index 0, and in E+ idf language, our Layer 0 field is called "Outside Layer", and our Layer 1 is called "Layer 2". This means that when the Layer index (i) is i = 0, we must specify the field name "Outside Layer" and when it is not zero,we must specify i = "Layer_" + str(i+1).

In [ ]:
def layer_from_index(i):
    if i == 0:
        return "Outside_Layer"
    else:
        return "Layer_" + str(i+1)

Now that we have the function that defines the idf construction object field names, we want to attribute material names to each field so the IDF can retrieve them from our materials list. We can develop a little function that applies the function above to any construction and returns a python dictionary with the correct field names associated to the index order of our construction array.

In [ ]:
list(enumerate(construction))

In [ ]:
def layers_dict(construction):
    return {layer_from_index(i):kind for i, kind in enumerate(construction)}

In [ ]:
layers_dict(construction)

We can then unpack the dictionary keys as the newidfobject function arguments, and values as argument values.

In [ ]:
idf1.newidfobject("CONSTRUCTION",
                  Name="New_wall_construction_0",
                  **layers_dict(construction))

In [ ]:
constructions

We have successfully added a construction object to our constructions library in the IDF File.

If we want to reset our idf to its original state either because we want to create new constructions or change some materials, we can repeat the steps of reading the idf with the respective fname, weather file,  read the materials idf library, replace the empty spaces among words in the materials library, and copy them to our zone idf.

In [ ]:
idf1 = IDF(fname1, weather_file)
materials = IDF(fname2, weather_file)
mats = materials.idfobjects["MATERIAL"] #Opaque materials
window_mats = materials.idfobjects["WINDOWMATERIAL:GLAZING"] #Window materials - Glazings
window_airgaps_mats = materials.idfobjects["WINDOWMATERIAL:GAS"] #Window materials - Airgaps
airgap_mats = materials.idfobjects["MATERIAL:AIRGAP"] #airgap materials for opaque constructions

full_mats = []
for i in [mats, window_mats, window_airgaps_mats, airgap_mats]:
    full_mats = np.append(full_mats, i) #appends all mats to a flat array
    
for mat in full_mats:
    mat.Name = mat.Name.replace(" ", "") #replaces spaces
    
for mat in full_mats:
    idf1.copyidfobject(mat) #copy mats to building idf 
    


In [ ]:
idf1.idfobjects["CONSTRUCTION"]

and here we have our original constructions and added materials from our materials library idf. But it is not done yet. Now we need to apply this construction to a specific building surface. To do this, we can start by creating again our construction, and by exploring the surfaces in our IDF file.

In [ ]:
idf1.newidfobject("CONSTRUCTION",
                  Name="New_wall_construction",
                  **layers_dict(construction))

In [ ]:
constructions = idf1.idfobjects["CONSTRUCTION"]

In [ ]:
constructions

In [ ]:
opaque_surfaces = idf1.idfobjects["BuildingSurface:Detailed"]
window_surfaces = idf1.idfobjects["FenestrationSurface:Detailed"]

In [ ]:
window_surfaces

In [ ]:
opaque_surfaces

Since we defined an exterior wall construction, we need to attribute the specific construction name to our exterior walls. From observing the objects in our surfaces, we see that there are two fields that can help us filter the exterior walls. An exterior wall as a boundary condition of "Outdoors", and a "Surface Type" of "Wall.

In [ ]:
for opaque_surface in opaque_surfaces:
    if opaque_surface["Outside_Boundary_Condition"] == "Outdoors" and opaque_surface["Surface_Type"] == "Wall":
        opaque_surface["Construction_Name"] = constructions[-1]["Name"] #we retrieve the name of the last item in constructions since we added it last, we could also write the construction name that we defined as a string

In [ ]:
opaque_surfaces

And we successfully changed our exterior walls construction to our defined construction. we can now rerun the idf and obtain the energy simulation results for our building with the new construction on exterior walls

# Hands-on exercise - finding the best construction solutions for each surface.

Imagining that we are planning build our created zone. We want to find the best possible combination of constructions on each surface type that yields the least energy consumption, with the least possible cost. However, if we have $3$ possible constructions for each exterior wall element, each floor element, and each window element we might have hundreds, if not thousands or millions of possible combinations. In this case, since we have $4$ exterior walls, $1$ floor, and $2$ windows, we have $3^7$ possible combinations which represents $2187$ possible solutions. if each simulation takes approximately 10 seconds, we would take around 6 hours to simulate all the possible combinations. Luckily, we can solve this optimization problem in less time with EPPY and python even for larger and more complex problems than this one. Let's start by resetting our idf file to the original, concatenating our materials library, and copying it to our building IDF.

In [ ]:
idf1 = IDF(fname1, weather_file)
materials = IDF(fname2, weather_file)
mats = materials.idfobjects["MATERIAL"] #Opaque materials
window_mats = materials.idfobjects["WINDOWMATERIAL:GLAZING"] #Window materials - Glazings
window_airgaps_mats = materials.idfobjects["WINDOWMATERIAL:GAS"] #Window materials - Airgaps
airgap_mats = materials.idfobjects["MATERIAL:AIRGAP"] #airgap materials for opaque constructions
constructions = idf1.idfobjects["CONSTRUCTION"]
full_mats = []
for i in [mats, window_mats, window_airgaps_mats, airgap_mats]:
    full_mats = np.append(full_mats, i) #appends all mats to a flat array
    
for mat in full_mats:
    mat.Name = mat.Name.replace(" ", "") #replaces spaces
    
for mat in full_mats:
    idf1.copyidfobject(mat) #copy mats to building idf 

Then we define the possible construction for each element type - walls, floors, and windows, and their respective cost/m2

In [ ]:
Wmaterials = [["Reboco-2cm", "TijoloFurado_15", "AirGap", "TijoloFurado_11", "Estuque_Claro_1.5"],
              ["Reboco-2cm", "TijoloFurado_11", "AirGap", "TijoloFurado_11", "Estuque_Claro_1.5"],
              ["Reboco-2cm", "TijoloFurado_15", "AirGap", "IsolamentoXPS-4cm", "TijoloFurado_11", "Estuque_Claro_1.5"]]

wall_costs = [25, 20, 45]

Fmaterials = [["PaineisdeMadeira_12", "Estuque_Claro_1.5"],
              ["Ceramicavidrada-1cm", "BetonilhadeAcentamento_8", "LajeBetao_15", "Estuque_Claro_1.5"],
              ["Ceramicavidrada-1cm", "BetonilhadeAcentamento_8", "LajeAligeirada_0.25", "Estuque_Claro_1.5"]]

floors_costs = [10, 25, 30]

Wndmaterials = [[2.69, 0.75, 0.8], [1.70, 0.38, 0.7], [1.25, 0.2, 0.7]]

wnd_costs = [50, 80, 100]


lets start by inserting these constructions in the new idf. We can start by setting new glazing system materials for each window since we haven't done it yet, and then define our constructions according to the selected materials

In [ ]:
for mat in enumerate(Wndmaterials):
    idf1.newidfobject("WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM",
                           Name='window_' + str(mat[0]),
                           UFactor = float(mat[1][0]),
                           Solar_Heat_Gain_Coefficient= float(mat[1][1]),
                           Visible_Transmittance = float(mat[1][2]))

In [ ]:
window_mats = idf1.idfobjects["WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM"]

In [ ]:
window_mats

In [ ]:
for window_mat in enumerate(window_mats):
        idf1.newidfobject("CONSTRUCTION",
                          Name = window_mat[1].Name,
                          Outside_Layer = window_mat[1].Name)

In [ ]:
constructions

The window constructions are set in the idf, now we can move on to the walls.

In [ ]:
for construction in enumerate(Wmaterials):
     idf1.newidfobject("CONSTRUCTION",
                       Name='EXT_WALL' + str(construction[0]),
                       **layers_dict(construction[1]))

In [ ]:
constructions

Now we do the same for the floor materials

In [ ]:
for construction in enumerate(Fmaterials):
    idf1.newidfobject("CONSTRUCTION",
                      Name='floor_' + str(construction[0]),
                      **layers_dict(construction[1]))

In [ ]:
constructions

Now that we have our constructions all set up, we can start elaborating the optimization function. We want an Energy function, that runs an idf with a specific combination of constructions, one for each surface. Thus, our variables are each surface construction. We want to optimize the exterior walls, floors, and windows. So our variables are the number of exterior wall surfaces, plus the number of floor surfaces, plus the number of window surfaces. Let's isolate the surface elements we want to iterate constructions on.

In [ ]:
opaque_surfaces = idf1.idfobjects["BuildingSurface:Detailed"]
window_surfaces = idf1.idfobjects["FenestrationSurface:Detailed"]

In [ ]:
window_surfaces

To isolate window surfaces we just need to gather all the idf objects in "FenestrationSurface:Detailed". However, to find walls and floors we need to search for them through all the opaque surfaces.

In [ ]:
opaque_surfaces

To search for the wall surfaces, we need to test all surfaces for their surface type (wall) and boundary condition(outdoors), if we only searched for walls, we could include interior walls if they existed.

In [ ]:
wall_surfaces = []
for opaque_surface in opaque_surfaces:
    if opaque_surface["Surface_Type"] == "Wall" and opaque_surface["Outside_Boundary_Condition"] == "Outdoors":
        wall_surfaces = np.append(wall_surfaces, opaque_surface)

In [ ]:
wall_surfaces

Now we do the same for the floors, but we just need to search through each surface type and not boundary conditions. If there was more than one floor, we would have to create an array for floor_surfaces and other array for ground floor surface, using boundary conditions as above.

In [ ]:
floor_surfaces = []
for opaque_surface in opaque_surfaces:
    if opaque_surface["Surface_Type"] == "Floor":
        floor_surfaces = np.append(floor_surfaces, opaque_surface)

In [ ]:
floor_surfaces

Now that we have isolated our surface variables, we need to write a function that receives a set with the length of the total number of surfaces that are variables with construction solutions to test for each surface, and returns the heating, cooling, and cost. Additionally, the energy output from the IDF comes in J, which is a hard number to read. However we can create a function that converts J to kWh/m2 based on the floor area of our building.

In [ ]:
def convert_to_kwh_m2(joules, area):
    return joules/(3600 * 1000 * area)

# Optimization function

In [ ]:
import time

def opt_output(xs):
    startTime = time.time() #start counting computing time
    column_names = []
    cost = []
    
    # Defining constructions based on input variables
    
    for i in range(len(wall_surfaces)):
        column_names = np.append(column_names, wall_surfaces[i]["Name"])
        wall_surfaces[i]["Construction_Name"] = "EXT_WALL" + str(xs[i])
        cost = np.append(cost, wall_surfaces[i].area * wall_costs[xs[i]])
        
    for i in range(len(floor_surfaces)):             
        column_names = np.append(column_names, floor_surfaces[i]["Name"])
        floor_surfaces[i]["Construction_Name"] = "floor_" + str(xs[i + len(wall_surfaces)])
        cost = np.append(cost, floor_surfaces[i].area * floors_costs[xs[i + len(wall_surfaces)]])
        
    for i in range(len(window_surfaces)):
        column_names = np.append(column_names, window_surfaces[i]["Name"])
        window_surfaces[i]["Construction_Name"] = "window_" + str(xs[i + len(wall_surfaces) + len(floor_surfaces)]) 
        cost = np.append(cost, window_surfaces[i].area * wnd_costs[xs[i+ len(wall_surfaces) + len(floor_surfaces)]])
    
    #Running the IDF and obtain inputs
    
    idf1.run() #run new idf
    eso = esoreader.read_from_path(r"C:\Users\GRA\OneDrive - Universidade de Lisboa\Desktop\IN+\C-Tech\EPPY\EPPY_Class\eplusout.eso")#Read result file
    data = pd.DataFrame(eso.data) #convert dict to dataframe
    data.columns = [eso.dd.variables[i][2] for i in data.columns]#set the column names from dict
    heating = convert_to_kwh_m2(data["Zone Ideal Loads Supply Air Sensible Heating Energy"].sum(), floor_surfaces[0].area) #Retrieve hourly heating energy
    cooling = convert_to_kwh_m2(data["Zone Ideal Loads Supply Air Total Cooling Energy"].sum(), floor_surfaces[0].area)  #Retrieve hourly cooling energy
    executionTime = time.time() - startTime #finish counting computing time
    df = pd.DataFrame(np.hstack([executionTime, xs, heating, cooling, np.sum(cost)])).T
    df.columns = np.hstack(("executionTime", column_names, "heating", "cooling", "cost"))
    df.to_csv(fname, mode='a', index=False, header=False) #write_to_file
    return [heating + cooling, np.sum(cost)]
    #return df

In [ ]:
import datetime
def time_str():
    now = datetime.datetime.now()
    now_str = str(now.year) + "_" + str(now.month) + "_" + str(now.day) + "_" + str(now.hour) + "_" + str(now.minute)
    return now_str

In [ ]:
xs_test = [2, 2, 2, 2, 2, 2, 2]

In [ ]:
fname = "test" + time_str() + '.csv'
opt_output(xs_test)

In [ ]:
wall_costs

In [ ]:
wall_surfaces[1].area

In [ ]:
floors_costs

In [ ]:
floor_surfaces[0].area

In [ ]:
wnd_costs

In [ ]:
window_surfaces[0].area

In [ ]:
! pip install Platypus-Opt

In [ ]:
from platypus import NSGAII, SPEA2, IBEA, Problem, Integer

nvars = len(wall_surfaces) + len(floor_surfaces) + len(window_surfaces)

problem_types = []
for i in range(nvars):
    problem_types = np.append(problem_types, Integer(0, 2))

In [ ]:
problem_types

In [ ]:
n_evals = 300
problem = Problem(nvars, 2)
problem.types[:] = problem_types
problem.function = opt_output

fname = "NSGA_Lisbon" + time_str() + '.csv'

with open(fname, 'a') as my_new_csv_file:
    algorithm = NSGAII(problem,
                       population_size= 40,
                        )
    algorithm.run(n_evals)

In [ ]:
x = [s.objectives[0] for s in algorithm.result]
y = [s.objectives[1] for s in algorithm.result]


df_NSGAII = pd.DataFrame([x, y]).transpose()
df_NSGAII.columns = ["heating & cooling", "total cost €"]

In [ ]:
problem = Problem(nvars, 2)
problem.types[:] = problem_types
problem.function = opt_output

fname = "SPEA2_Lisbon" + time_str() + '.csv'

with open(fname, 'a') as my_new_csv_file:
    algorithm = SPEA2(problem,
                      population_size= 40)
    algorithm.run(n_evals)

In [ ]:
x = [s.objectives[0] for s in algorithm.result]
y = [s.objectives[1] for s in algorithm.result]

df_SPEA2 = pd.DataFrame([x, y]).transpose()
df_SPEA2.columns = ["heating & cooling", "total cost €"]

In [ ]:
problem = Problem(nvars, 2)
problem.types[:] = problem_types
problem.function = opt_output

fname = "IBEA_Lisbon" + time_str() + '.csv'

with open(fname, 'a') as my_new_csv_file:
    algorithm = IBEA(problem,
                     population_size= 40)
    algorithm.run(n_evals)

In [ ]:
x = [s.objectives[0] for s in algorithm.result]
y = [s.objectives[1] for s in algorithm.result]

df_IBEA = pd.DataFrame([x, y]).transpose()
df_IBEA.columns = ["heating & cooling", "total cost €"]

In [ ]:
! pip install plotly

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
dfs = {"IBEA" : df_IBEA, "SPEA2": df_SPEA2, "NSGAII" : df_NSGAII}

for i in dfs:
    fig = fig.add_trace(go.Scatter(#data_frame = dfs[i],
                                    x = dfs[i]["heating & cooling"],
                                    y = dfs[i]["total cost €"],
                                    name = i,
                                    opacity = 0.7,
                                    #line = None,
                                    mode = "lines",
                                    marker = dict(size = 0),
                                    #alphahull =0,
                                    showlegend= True))
    
fig.show()

This work was supported by international funds through Fundo Europeu de Desenvolvimento Regional (FEDER) with reference <i> POCI-01-0247-FEDER-045919 </i>, national funds through PhD grant under contract of FCT with reference <i>2021.04849.BD.</i><p>
If you use this notebook for research purposes please cite: <p>
<i>G. Araujo, L. Santos, A. Leitão, and R. Gomes, “Ad-based surrogate models for simulation and optimization of large urban areas,” in 27th International Conference of the Association for Computer-Aided Architectural Design Research in Asia (CAADRIA 2022)